In [1]:
from dm_control import suite

In [2]:
env = suite.load(domain_name="swimmer", task_name="swimmer6")

In [3]:
print(env)

In [10]:
from dm_control import suite
from dm_control import viewer
import numpy as np

# env = suite.load(domain_name="humanoid", task_name="stand")
env = suite.load(domain_name="swimmer", task_name="swimmer6")
action_spec = env.action_spec()
print(action_spec)

# Define a uniform random policy.
def random_policy(time_step):
#     del time_step  # Unused.
    print(time_step)
    
    action = np.random.uniform(low=action_spec.minimum,
                           high=action_spec.maximum,
                           size=action_spec.shape)
    print(action)
    return action
# time_step = env.reset()

# Launch the viewer application.
viewer.launch(env, policy=random_policy)

BoundedArray(shape=(5,), dtype=dtype('float64'), name=None, minimum=[-1. -1. -1. -1. -1.], maximum=[1. 1. 1. 1. 1.])
TimeStep(step_type=<StepType.FIRST: 0>, reward=None, discount=None, observation=OrderedDict([('joints', array([ 0.58728508, -0.44662536,  0.29398981, -0.27563479,  0.57042761])), ('to_target', array([0.12629341, 0.1936908 ])), ('body_velocities', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]))]))
[0.05937119 0.69068594 0.12978417 0.21802183 0.06221048]
TimeStep(step_type=<StepType.MID: 1>, reward=0.6167314968194839, discount=1.0, observation=OrderedDict([('joints', array([ 0.58240431, -0.43704721,  0.28649061, -0.27296249,  0.5714522 ])), ('to_target', array([0.12639203, 0.19381687])), ('body_velocities', array([-0.00281281, -0.01098553,  0.02619254,  0.00450647, -0.006907  ,
       -0.27983416,  0.00376855,  0.00157187,  0.31828799, -0.00369028,
        0.00494007, -0.15032654,  0.00146291,  0.00578861,  0.01779418,
       -0.0004

TimeStep(step_type=<StepType.MID: 1>, reward=0.6444140045920271, discount=1.0, observation=OrderedDict([('joints', array([ 0.98721006, -0.96058428,  0.80168069, -0.64647406,  0.70967817])), ('to_target', array([0.10943095, 0.19522704])), ('body_velocities', array([-4.01439770e-03,  4.26787433e-02,  4.25527419e-01,  5.78091711e-03,
        4.46217909e-02,  3.17968506e-01, -1.17431142e-03,  1.72785605e-02,
       -8.23843587e-01, -8.22306071e-04, -1.67366395e-02,  8.21396631e-01,
        5.16516592e-03, -3.85942042e-02, -5.70445137e-01, -1.42456912e-02,
       -5.12268447e-02,  2.93048585e-01]))]))
[ 0.80319139 -0.46199451  0.77453987  0.01928206 -0.46320522]
TimeStep(step_type=<StepType.MID: 1>, reward=0.649640543146051, discount=1.0, observation=OrderedDict([('joints', array([ 0.99722472, -1.00446948,  0.85507304, -0.68226518,  0.71972287])), ('to_target', array([0.11047243, 0.19301829])), ('body_velocities', array([-1.76855603e-02,  6.47765634e-02, -3.86065317e-02,  1.26199528e-02,
  

       -0.02561026, -0.0446703 ,  1.31621078]))]))
[ 0.25397532 -0.87831495  0.83582928 -0.46569471 -0.46291728]
TimeStep(step_type=<StepType.MID: 1>, reward=0.6459343122395287, discount=1.0, observation=OrderedDict([('joints', array([ 1.11571904, -1.02143351,  1.04527412, -1.00913399,  0.94103883])), ('to_target', array([0.09843579, 0.20053824])), ('body_velocities', array([-0.01316448,  0.01262042, -0.06642039,  0.01176742,  0.01438874,
       -0.09515863,  0.00281216,  0.02160676, -0.12910191, -0.00058841,
        0.0028226 ,  0.47857308,  0.01291824, -0.01924723, -0.56729669,
       -0.01831428, -0.04469939,  0.54145702]))]))
[ 0.34115544 -0.94885987 -0.7415305  -0.56203542  0.2620907 ]
TimeStep(step_type=<StepType.MID: 1>, reward=0.6471727492345904, discount=1.0, observation=OrderedDict([('joints', array([ 1.11476287, -1.02493504,  1.0616593 , -1.04103786,  0.97955528])), ('to_target', array([0.09880901, 0.19998224])), ('body_velocities', array([-0.00697187,  0.02021458,  0.088812

TimeStep(step_type=<StepType.MID: 1>, reward=0.6412693236929228, discount=1.0, observation=OrderedDict([('joints', array([ 0.83046405, -0.809366  ,  1.03110907, -1.12932611,  1.04287747])), ('to_target', array([0.11408234, 0.19353419])), ('body_velocities', array([ 0.01852398, -0.07834323,  1.32491827, -0.01813327, -0.01761466,
       -1.4378644 ,  0.02510009,  0.02676077,  0.49484721,  0.0143245 ,
        0.01344461,  0.17633138, -0.02272297,  0.01072432,  0.25841476,
        0.01588035,  0.03619331, -0.49142207]))]))
[-0.33711868 -0.27883892  0.85753803 -0.36195828  0.61484615]
TimeStep(step_type=<StepType.MID: 1>, reward=0.6400706830737939, discount=1.0, observation=OrderedDict([('joints', array([ 0.74892379, -0.75814073,  1.02381726, -1.12347752,  1.02503332])), ('to_target', array([0.11907374, 0.19088692])), ('body_velocities', array([ 0.01183591, -0.0657917 ,  1.40784942, -0.02379507, -0.008319  ,
       -1.27774764,  0.02213684,  0.02152582,  0.25389447,  0.01903752,
        0.0

TimeStep(step_type=<StepType.MID: 1>, reward=0.6147073822268265, discount=1.0, observation=OrderedDict([('joints', array([ 0.57152946, -0.52358884,  0.8011704 , -0.8718107 ,  0.75504923])), ('to_target', array([0.12534713, 0.19516395])), ('body_velocities', array([ 0.00692725, -0.05534403,  1.43715926, -0.01078091, -0.01142619,
       -1.47534614,  0.02771265,  0.02159723,  0.65096975, -0.00369008,
        0.01850175,  0.31670964, -0.0273419 , -0.00304193,  0.01230669,
        0.01946819,  0.01694452, -0.83824825]))]))
[ 0.3796627  -0.14605943  0.68983123  0.59100956 -0.94553872]
TimeStep(step_type=<StepType.MID: 1>, reward=0.6148313924573656, discount=1.0, observation=OrderedDict([('joints', array([ 0.50506461, -0.47876016,  0.7977507 , -0.86788028,  0.70602069])), ('to_target', array([0.12966729, 0.19227907])), ('body_velocities', array([-6.89783421e-04, -3.44621060e-02,  8.22175689e-01, -1.19252267e-02,
       -9.93445650e-03, -8.26589565e-01,  2.40871761e-02,  4.72843044e-03,
     

...

TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.67761551,  0.30633544, -0.64905219,  0.8381374 , -0.43423868])), ('to_target', array([-0.03909265, -0.01098071])), ('body_velocities', array([-7.80030825e-03,  2.47172487e-02, -1.01860140e+00,  5.29337897e-03,
       -7.78318526e-03,  8.76077580e-01, -1.76327597e-02,  4.19264535e-03,
       -4.28640733e-01,  3.50003070e-03,  5.63650351e-03, -6.01495451e-02,
       -2.03455384e-02, -1.06770465e-03,  5.77764903e-01,  2.03266891e-02,
       -2.16822421e-02, -1.29083821e+00]))]))
[-0.0671637   0.64262506  0.83836844 -0.66217185 -0.89192834]
TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.72321749,  0.27279043, -0.63073493,  0.85216164, -0.49925413])), ('to_target', array([-0.03702347, -0.01249454])), ('body_velocities', array([-0.00749825,  0.01205432, -0.73189153,  0.00486155, -0.01021907,
        0.49857896,  0.0019005

TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.86281784,  0.29111098, -0.56781699,  0.49192004,  0.0122155 ])), ('to_target', array([-0.02808663, -0.01645927])), ('body_velocities', array([-4.89601044e-03, -6.58992475e-03,  9.25242254e-01,  4.66438685e-03,
        3.45782772e-02, -8.58774097e-01, -1.72586833e-04,  1.94605374e-02,
        1.11394696e+00, -8.31321322e-03, -1.36397223e-02, -9.85103633e-01,
        2.59482493e-02, -3.13601029e-02,  7.39366072e-02, -1.86383779e-02,
       -3.16295686e-02,  8.10101339e-01]))]))
[-0.16896811 -0.34261274  0.96459868  0.46418473 -0.71631795]
TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.82010987,  0.33037961, -0.61553972,  0.53302854,  0.00296194])), ('to_target', array([-0.02989757, -0.01557612])), ('body_velocities', array([ 6.52349867e-04, -5.41942446e-03,  7.35287783e-01, -8.57839760e-03,
        2.27090627e-02, -4.

TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.46313325,  0.45988134, -0.77469115,  1.01477264, -0.5866775 ])), ('to_target', array([-0.04557649, -0.00454233])), ('body_velocities', array([ 5.01123140e-03,  1.26394536e-02, -1.29860703e+00, -1.10558795e-03,
       -1.99386685e-02,  1.38652084e+00, -1.81524648e-02,  1.33933561e-02,
       -1.07622594e+00,  1.53334651e-02,  3.45145780e-02,  1.56213746e-02,
       -1.06265540e-02,  5.85701313e-03,  9.52456410e-01,  6.85898173e-03,
       -2.73690578e-02, -1.17221179e+00]))]))
[-0.04462316  0.19980795  0.50907116 -0.71434849 -0.81738993]
TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 0.53105807,  0.39163009, -0.7352791 ,  1.03689147, -0.657938  ])), ('to_target', array([-0.0436107 , -0.00627282])), ('body_velocities', array([ 0.00495966,  0.00682168, -0.92028791, -0.00444623, -0.01993347,
        1.03709053, -0.0043680

TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 1.09559644, -0.22765608, -0.25104948,  0.72163167, -0.47800696])), ('to_target', array([-0.03013019, -0.02130861])), ('body_velocities', array([ 0.01811722, -0.00063822,  0.33278995,  0.00068635, -0.00160599,
       -0.01155557, -0.00670232, -0.00127925,  0.16592593, -0.01343984,
       -0.00496511, -0.01541612,  0.00735378,  0.00464185, -0.40289592,
        0.00919613,  0.01677119,  0.26169859]))]))
[ 0.60137306  0.42840839 -0.18608895  0.33058075 -0.06294677]
TimeStep(step_type=<StepType.MID: 1>, reward=1.0, discount=1.0, observation=OrderedDict([('joints', array([ 1.08856841, -0.21480794, -0.26674068,  0.71679341, -0.46133546])), ('to_target', array([-0.03084291, -0.02120929])), ('body_velocities', array([ 0.00468581,  0.00350082,  0.02007692,  0.01111873, -0.00163857,
       -0.12619207, -0.00820336,  0.00211411,  0.51160839, -0.0182467 ,
       -0.00686568, -0.29799163,  0.0

KeyboardInterrupt: 

In [11]:
print(env.observation_spec())

OrderedDict([('joints', Array(shape=(5,), dtype=dtype('float64'), name='joints')), ('to_target', Array(shape=(2,), dtype=dtype('float64'), name='to_target')), ('body_velocities', Array(shape=(18,), dtype=dtype('float64'), name='body_velocities'))])


In [2]:
from dm_control import suite
from dm_control import viewer
import numpy as np

# env = suite.load(domain_name="humanoid", task_name="stand")
env = suite.load(domain_name="swimmer", task_name="swimmer6",visualize_reward = True)
action_spec = env.action_spec()
obs_spec = env.observation_spec()
# 1 eposode
step_data = env.reset()
reward=0
counter=0
while not step_data.last():
    action = np.random.uniform(low=action_spec.minimum,high=action_spec.maximum,size=action_spec.shape)
    step_data = env.step(action)
    counter+=1
    reward += step_data.reward
    print(counter)
    print(step_data.reward)
    
# viewer.launch(env, policy=action)
#step_type, reward , discount, observation

1
0.01067655671903735
2
0.010671217090511515
3
0.010666076234009173
4
0.010658192071318073
5
0.010647767501726411
6
0.010639985158866362
7
0.010634791936017483
8
0.01063371701078075
9
0.010634026844399989
10
0.010631836729660556
11
0.010631362367482319
12
0.01063194520794541
13
0.010628994945082555
14
0.010625500966387872
15
0.010626593112488097
16
0.01062970887382277
17
0.010633353998652003
18
0.010636927023777328
19
0.010639222789850234
20
0.010639671615276715
21
0.010638360302955865
22
0.010635279431818296
23
0.010630124352335254
24
0.010624579146068461
25
0.010622696747422552
26
0.010625385402703195
27
0.010633102362926106
28
0.010643884981752052
29
0.010654685630149754
30
0.010664401784178799
31
0.010674914507820144
32
0.010684703408252913
33
0.010688976367430419
34
0.01069041505946663
35
0.010694821644100155
36
0.010699537545581091
37
0.010703168480767097
38
0.01070336294525796
39
0.010703105512355843
40
0.010709266937040631
41
0.010717651004260904
42
0.01072247438970401
43
0.010

384
0.01055306566864977
385
0.010552770981706056
386
0.010554380149693643
387
0.010552872626882313
388
0.010546216023356841
389
0.01053807360165051
390
0.010533723140672066
391
0.010525680723619727
392
0.010499404206134492
393
0.01047455284945012
394
0.010455813034364514
395
0.010432242069404766
396
0.010415912879258337
397
0.010419568578200221
398
0.01042740378244945
399
0.010423479435953651
400
0.010419706372402469
401
0.010416020530539808
402
0.010412397809720066
403
0.010403011145510472
404
0.010385530047791218
405
0.010372704923182884
406
0.010366385753714614
407
0.010372660739845267
408
0.010375715522168111
409
0.010370559240511973
410
0.01036967416105864
411
0.010376069294913257
412
0.010389439708930765
413
0.010400206985758994
414
0.010404189730808838
415
0.010398201029822942
416
0.010387166812703009
417
0.010370251673979149
418
0.010349650823385756
419
0.010337329855022072
420
0.010332581886570406
421
0.010341384435083612
422
0.010349963728045244
423
0.010356382112627127
424
0

778
0.010237321895235015
779
0.010198683350164852
780
0.010150391180740792
781
0.010116210175849315
782
0.010088775235786003
783
0.010064043959598371
784
0.010045556052805394
785
0.01002913463261093
786
0.010016899034048947
787
0.010009977271450192
788
0.010010417212074624
789
0.010012134573780442
790
0.010015700196478254
791
0.010028801464059921
792
0.010036163176257005
793
0.010023788278909096
794
0.010004011523652807
795
0.009981420144401097
796
0.009958838304975555
797
0.009945358550613808
798
0.009942924933172535
799
0.009952090056950396
800
0.009971136502473579
801
0.009993700807106314
802
0.010022724279541236
803
0.010044082072586117
804
0.010054863653219932
805
0.010059396242840413
806
0.010059678881330112
807
0.010065994964805156
808
0.010067744210642677
809
0.010063671074790103
810
0.010050532611592269
811
0.010026194513726114
812
0.010003566430684745
813
0.009993249345195725
814
0.009995892961731295
815
0.010011977081617232
816
0.010030817227824255
817
0.01005334896938399
81